In [ ]:
import numpy as np
import torch as ch
from pathlib import Path
from unlearning.auditors.utils import (
    loader_factory,
 )
import yaml
import numpy as np
import torch as ch
import matplotlib.pyplot as plt
from pathlib import Path

from unlearning.auditors import ulira

from unlearning import BASE_DIR, LOG_DIR, ULIRA_BASE_DIR



def read_yaml(yaml_file):
    with open(yaml_file, "r") as f:
        config = yaml.safe_load(f)
    return config


def load_model(path, model_factory, ds_name):
    model = model_factory(ds_name)
    loaded_model = ch.load(path)
    first_key = list(loaded_model.keys())[0]
    if "model" in first_key:
        model.load_state_dict(loaded_model)

    else:
        # add ".model" to each key in k,vs
        loaded_model = {f"model.{k}": v for k, v in loaded_model.items()}
        model.load_state_dict(loaded_model)
    return model


#unlearnings_per_model = 20
unlearnings_per_model = 40

In [ ]:
ULIRA_BASE_DIR

In [ ]:
# compute margins from logits


In [ ]:
# compute margins from the logits
# ORACLE MODEL margin computation
dataset_name = "CIFAR10"
model_dir_name = "final_models"
model_dir_name = "resnet_long"

train_f = ULIRA_BASE_DIR / model_dir_name# / "train_margins_all.pt"
val_f = ULIRA_BASE_DIR / model_dir_name #/ "val_margins_all.pt"



def load_individual_margins(dir_, max_count=300):
    """ load individual margins from a directory, as 1 mega tensor"""

    def get_num(p):
        return int(p.name.split("_")[2].split(".")[0])

    train_margins = list(dir_.glob("train_margins_*.pt"))
    val_margins = list(dir_.glob("val_margins_*.pt"))

    val_margins = sorted(val_margins, key=get_num)
    train_margins = sorted(train_margins, key=get_num)
    # load and concatenate
    all_train_margins = []
    all_val_margins = []
    for i in range(max_count):
        if i % 10 == 0:
            print(f"loading {i}")
        all_train_margins.append(ch.tensor(ch.load(train_margins[i])))
        all_val_margins.append(ch.tensor(ch.load(val_margins[i])))

    return all_train_margins, all_val_margins



if model_dir_name == "final_models":
    ds_name = "CIFAR10"
    #forget_masks = ulira.get_ulira_forget_mask(ds_name, class_5_range=1000, overwrite=False )print(f"original training_masks- shape {training_masks.shape}")
    # which points to forget
    ulira_train_all_margins, ulira_val_all_margins = ulira.load_all_ulira_margins(
        ds_name)

    all_oracle_margins_ulira = ch.cat([
        ch.tensor(ulira_train_all_margins.T),
        ch.tensor(ulira_val_all_margins.T)
    ]).T

else:
    model_count = 300

    ulira_train_all_margins, ulira_val_all_margins = load_individual_margins(
        train_f, model_count)
    # stakc em
    #print(ulira_train_all_margins[0].shape)

    ulira_train_all_margins = ch.stack(ulira_train_all_margins)
    # squash the margins into a single tensor
    ulira_val_all_margins = ch.stack(ulira_val_all_margins)

    all_oracle_margins_ulira = ch.cat([
        ch.tensor(ulira_train_all_margins).T,
        ch.tensor(ulira_val_all_margins).T
    ]).T

    #ch.cat([all_train_margins, all_val_margins])
all_oracle_margins_ulira.shape

In [ ]:
type(ulira_train_all_margins[0])
ulira_val_all_margins.shape

Issues that could be :
1. incorrectly reading the forget masks
2. incorrectly reading the align masks
3. incorectly reading the val margins

In [ ]:
training_masks = ulira.get_ulira_training_masks()



# Repeat each row n times
#result = all_oracle_margins_ulira.repeat(unlearnings_per_model, 1).view(-1, all_oracle_margins_ulira.size(1))

dont_repeat_them = False
print(f"do tiling for masks and for oracle margins")
if dont_repeat_them:
    repeated_training_masks = np.repeat(training_masks, unlearnings_per_model, axis=0)

    repeated_all_oracle_margins_ulira = all_oracle_margins_ulira.clone()
    repeated_all_oracle_margins_ulira = repeated_all_oracle_margins_ulira.repeat_interleave(unlearnings_per_model, dim=0)
    #
else:

    non_repeated_training_masks = training_masks.copy()
    non_repeated_all_oracle_margins_ulira = all_oracle_margins_ulira.clone()

    training_masks = np.repeat(training_masks, unlearnings_per_model, axis=0)
    all_oracle_margins_ulira = all_oracle_margins_ulira.repeat_interleave(unlearnings_per_model, dim=0)
    #repeated_all_oracle_margins_ulira = all_oracle_margins_ulira.clone()

print(f"training_masks - shape {training_masks.shape}")

print(f"all_oracle_margins_ulira - shape {all_oracle_margins_ulira.shape}")




In [ ]:
from unlearning.auditors.ulira_plans import get_ulira_forget_masks, load_ulira_forget_masks, load_all_ulira_forget_masks
forget_masks = load_all_ulira_forget_masks()
#forget_masks_40 = load_ulira_forget_masks(original_model_count = 256)
#forget_masks = np.concatenate(forget_masks_40)
# save the forget masks
print(f"loadded - {forget_masks.shape}")
def get_forgettable_indices(forget_masks):
    """ extract out the forgettable indices from the forget_masks"""
    forgettable_indices = set([])
    #for forget_group in forget_masks:
    for row in forget_masks:
        l = set(row.nonzero()[0])
        forgettable_indices= forgettable_indices.union(l)
    # this value should be 1000
    #if len(forgettable_indices) != 1000:
    #    raise ValueError("forget_points should have 1000 points")
    # for each of these points, compute a ulira score using the margins
    forgettable_indices= list(forgettable_indices)
    return forgettable_indices
forgettable_indices = get_forgettable_indices(forget_masks)

In [ ]:

if False:
    base_dir = Path("/n/home04/rrinberg/data_dir__holylabs/unlearning/precomputed_models/ULIRA_clean")
    ga_name = 'CIFAR10____benchmark_GA_wrapper__dataset=CIFAR10__num_epochs=5__learning_rate=0.001__batch_size=64__forget_batch_size=64'
if False:
    gd_name = 'CIFAR10____benchmark_GD_wrapper__dataset=CIFAR10__num_epochs=5__learning_rate=0.001__forget_batch_size=64'


    scrub_name = "CIFAR10____scrub__dataset=CIFAR10__num_epochs=10__learning_rate=0.01__forget_batch_size=32__beta=0.999__retain_batch_size=64__maximization_epochs=3"

    ga_name = "CIFAR10____benchmark_GA_wrapper__dataset=CIFAR10__num_epochs=5__learning_rate=0.001__batch_size=64__forget_batch_size=64"

def sort_by_last_num(s):

    last = str(s).split("__")[-1]
    return int(last[:-len(".pt")])


base_dir = ULIRA_BASE_DIR / "results"
ga_name = "CIFAR10____benchmark_GA_wrapper__dataset=CIFAR10__num_epochs=5__learning_rate=0.001__batch_size=64__forget_batch_size=64"
ga_uliras_pts = sorted(list((base_dir / ga_name / "ulira").glob("*.pt")), key=sort_by_last_num)


gd_name = "CIFAR10____benchmark_GD_wrapper__dataset=CIFAR10__num_epochs=5__learning_rate=0.001__forget_batch_size=64"

scrub_name = 'CIFAR10____scrub__dataset=CIFAR10__num_epochs=10__learning_rate=0.01__forget_batch_size=32__beta=0.999__retain_batch_size=64__maximization_epochs=3'

gd_uliras_pts = sorted(list((base_dir / gd_name / "ulira").glob("*.pt")),
                        key=sort_by_last_num)

scrub_uliras_pts = sorted(list(
    (base_dir / scrub_name / "ulira").glob("*.pt")),
                            key=sort_by_last_num)

print(f"ga-  {len(ga_uliras_pts)}")
print(f"gd { len(gd_uliras_pts)}")
print(f"scrub - {len(scrub_uliras_pts)}") 

In [ ]:
for name, pts in [("GD", gd_uliras_pts), ("GA", ga_uliras_pts),
                  ("Scrub", scrub_uliras_pts)]:
    print(name)
    nums = []
    for p in pts:
        num = int(p.name.split("__")[-1][:-len(".pt")])
        nums.append(num)
    expected = list(range(0, 2000, 40))
    missing = set(expected) - set(nums)
    print(sorted(list(missing)))
    indices = sorted(np.array(list(missing)) / 20)
    print(f"indices {indices}")
# scrub - need to do : 0 (0)
# GD - 350 (7)


In [ ]:
8,30,54,56,58,66,68,78,80,82,84,96

In [ ]:
ga_uliras_pts[:4]

In [ ]:
def load_and_merge(uliras_pts):
    all_margins = []
    for pt in uliras_pts:
        margin = ch.load(pt)
        all_margins.append(margin)
    # repeat the marings by the number of unlearnings

    ret = ch.cat(all_margins, dim=0)
    #ret = ret.repeat_interleave(unlearnings_per_model, dim=0)
    return ret

ga_uliras_pts_all = load_and_merge(ga_uliras_pts)

#gd_uliras_pts_all = load_and_merge(gd_uliras_pts)
#scrub_uliras_pts_all = load_and_merge(scrub_uliras_pts)

print(f"unlearned models : {ga_uliras_pts_all.shape}")
print(f"oracles - {all_oracle_margins_ulira.shape}")

print(f"doing scrub_uliras_pts_all Descent")
all_unlearned_margins_ulira = ga_uliras_pts_all

all_oracle_margins_ulira = all_oracle_margins_ulira[:all_unlearned_margins_ulira.shape[0]]



In [ ]:
nums = []
for p in ga_uliras_pts:
    i = p.name.split("__")[-1][:-len(".pt")]
    num = int(i)
    nums.append(num)

expected = list(range(0, 2000, unlearnings_per_model ))
a = list(set(expected) - set(nums))
print(f"missing : {sorted(a)}")

In [ ]:
def get_forget_mask_and_training_mask(index):
    """ get the forget mask and training mask for a given index"""
    original_model_i = index // unlearnings_per_model
    # Note: forget_masks is a concatenation of all the forget masks,
    return training_masks[original_model_i], forget_masks[index]



In [ ]:
# get 200 points from class 5 that are not from the forgettable_points
#forgettable_indices
def get_test_points_for_ulira(forget_indices, forgettable_indices, num_test_points=200, seed = 42):
    """
    get 200 points forgettable points from class 5 that are not in the forget_indices
    """
    np.random.seed(seed)
    # copy forgettable_indices
    class_5 = forgettable_indices.copy()
    class_5 = np.array(list(set(class_5) - set(forget_indices)))
    np.random.shuffle(class_5)
    return list(class_5)[:num_test_points]

def gaussian_pdf(x, mean, std):
    return 1 / (std * np.sqrt(2 * np.pi)) * np.exp(-0.5 * ((x - mean) / std) ** 2)


def single_ulira(
        all_unlearned_margins: ch.Tensor,  # T x n
        all_test_margins: ch.Tensor,  # T x n
        unlearned_model_margin: float,
        sample_ind,
        threshold=0.5,
        plot=False,
        verbose=False,
        drop_outliers=False):
    """
    Compute the ULIRA (Unlearning Likelihood Ratio) results for each sample. for a single unlearned model.

    Note: 
        1: (x,y) is likely a member of training (unlearned model)
        0: (x,y) is likely a member of the oracle (oracle model)

    Args:
        all_unlearned_margins (ch.Tensor): Tensor of shape T x n containing the margins of all unlearned models.
        all_oracle_margins (ch.Tensor): Tensor of shape T x n containing the margins of all oracle models.
        hold_out_model (int): Index of the hold-out model.
        threshold (float, optional): Threshold value for the likelihood ratio. Defaults to 0.5.

    Returns:
        np.ndarray: Array of ULIRA results for each sample, where 1 indicates the likelihood ratio is above the threshold, and 0 otherwise.
    """
    #
    test_margins = all_test_margins[:, sample_ind].cpu().numpy()

    unlearned_margins = all_unlearned_margins[:, sample_ind].cpu().numpy()

    #
    #unlearned_model_margin = unlearned_margins[hold_out_model]
    #other_unlearned_models = np.delete(unlearned_margins, hold_out_model)

    # fit gaussians
    def fit_gaussian(margins, drop_num=5):
        m = sorted(margins)
        # remove the top and bottom 3%
        if drop_outliers:
            m = m[drop_num:-drop_num]

        mean, std = np.mean(m), np.std(m)
        return mean, std

    test_mean, test_std = fit_gaussian(test_margins)
    unlearned_mean, unlearned_std = fit_gaussian(unlearned_margins)

    #print(f"aaa ({np.round(oracle_mean,2)}, {np.round(oracle_std,2)}) vs ({np.round(unlearned_mean,2)}, {np.round(unlearned_std,2)})")
    # compute LIRA
    unlearned_model_margin = float(unlearned_model_margin)

    oracle_prob = gaussian_pdf(unlearned_model_margin, test_mean, test_std)
    unlearned_prob = gaussian_pdf(unlearned_model_margin, unlearned_mean,
                                  unlearned_std)

    likelihood_ratio = unlearned_prob / (unlearned_prob + oracle_prob)

    # print(f"likelihood_ratio - {likelihood_ratio} : type - {type(likelihood_ratio)}")

    likelihood_ratio = float(likelihood_ratio)
    if verbose:
        print(f"unlearned_margins shape - {unlearned_margins.shape}")
        print(f"oracle_margins shape - {test_margins.shape}")
        # print(f"oracle_margins - {test_margins}")

    if plot:
        # plot gaussians
        plt.xlim(-20, 20)
        x = np.linspace(-20, 20, 1000)

        plt.hist(test_margins, bins=50, alpha=0.5, label="test", density=True)
        plt.hist(unlearned_margins,
                 bins=50,
                 alpha=0.5,
                 label="unlearned",
                 density=True)
        oracle_pdf = gaussian_pdf(x, test_mean, test_std)
        unlearned_pdf = gaussian_pdf(x, unlearned_mean, unlearned_std)
        num_pts = len(unlearned_margins)
        num_unique_pts = len(np.unique(unlearned_margins))

        plt.title(
            f"sample index: {sample_ind} | likelihood ratio: {round(likelihood_ratio,2):.2f} = {round(unlearned_prob,2)} / ({round(unlearned_prob,2)} + {round(oracle_prob,2)})\n num pts: {num_pts} | num unique pts: {num_unique_pts}"
        )
        plt.plot(x, oracle_pdf, label="oracle")
        plt.plot(x, unlearned_pdf, label="unlearned")
        plt.axvline(unlearned_model_margin, color="black", linestyle="--")
        plt.legend()
        plt.show()
    # save result
    return float(likelihood_ratio > threshold)



#####
def get_training_models_with_and_without(masks, index):
    """
    return indices of models with and without the point
    """
    m = masks[:, index].T

    with_index, without_index = np.where(m == 1)[0], np.where(m == 0)[0]
    return with_index, without_index



def get_models_with_pt_forgotten(masks, forget_mask, sample_index):
    """
    find the indices where the point is in the plan and in the forget mask
    """

    models_with = []
    models_inds_with, _ = get_training_models_with_and_without(
    masks, sample_index)

    for model_ind in models_inds_with:
        if forget_mask[model_ind, sample_index] == 1:
            models_with.append(model_ind)
    return np.array(models_with)

def get_models_with_pt_retained(masks, forget_mask, sample_index):
    """
    find the indices where the point is in the plan and in the forget mask
    """

    models_with = []
    models_inds_with, _ = get_training_models_with_and_without(
    masks, sample_index)

    for model_ind in models_inds_with:
        if forget_mask[model_ind, sample_index] == 0:
            models_with.append(model_ind)
    return np.array(models_with)
#####

def do_ulira_on_one_model(indices,
                          correct_ulira_label,
                          target_margins,
                          shadow_unlearning_margins,
                          shadow_oracle_margins,
                          forget_masks,
                          training_masks,
                          plot_index=-1, verbose=  False ):
    """

    """
    correct_ulira = 0
    ulira_labels = []
    for i, sample_index in enumerate(indices):
        if i % 50 == 0:
            print(f"i - {i}/ {len(indices)} = {round(correct_ulira *100. / (i+1),2)}")
        # get models with x forgotten and models that never saw x
        models_with_x_forgotten = get_models_with_pt_forgotten(
            training_masks, forget_masks, sample_index)
        _, models_without_x = get_training_models_with_and_without(
            training_masks, sample_index)

        num_models_with_x_forgotten = len(models_with_x_forgotten)
        num_models_without_x = len(models_without_x)
        if verbose:
            print(f"num_models_with_x_forgotten - {num_models_with_x_forgotten}")
            print(f"num_models_without_x - {num_models_without_x}")

        max_models = 300

        min_models = min(num_models_with_x_forgotten, num_models_without_x)
        min_models = min(max_models, min_models)
        #
        #print(f"min models - {min_models    }")
        #min_models = -1
        models_with_x_forgotten = models_with_x_forgotten[:min_models]
        models_without_x = models_without_x[:min_models]

        # the margin of the target model on sample index
        target_margin = target_margins[sample_index]

        # we want to do a lira test between unlearned models that saw x
        # and unlearned models that did not see x

        ulira_label = single_ulira(
            shadow_unlearning_margins[models_with_x_forgotten],
            shadow_unlearning_margins[models_without_x],
            target_margin,
            sample_index,
            plot=(i == plot_index), verbose= verbose)


        if False:
            ulira_label = single_ulira(
                shadow_unlearning_margins[models_with_x_forgotten],
                shadow_oracle_margins[models_without_x],
                target_margin,
                sample_index,
                plot=(i == plot_index))
        ulira_labels.append(ulira_label)
        if ulira_label == correct_ulira_label:
            correct_ulira += 1
        plot_it_all = False# correct_ulira_label==0 #True
        if plot_it_all:
            if ulira_label == correct_ulira_label:
                print(f"plotting good boy - expecting {correct_ulira_label}, where 1 = forget, 0 = retain")

            else:
                # plot it :

                print(f"plotting wrong boy - expecting {correct_ulira_label}, where 1 = forget, 0 = retain")

            ulira_label = single_ulira(
                    shadow_unlearning_margins[models_with_x_forgotten],
                    shadow_unlearning_margins[models_without_x],
                    target_margin,
                    sample_index,
                    plot=True, verbose= verbose)

    return correct_ulira



def ulira_paper(all_unlearned_margins_ulira: ch.Tensor,
                all_oracle_margins_ulira: ch.Tensor,
                forget_masks,
                training_masks,
                plot_index=-1,
                verbose=False,
                max_models=None):
    """
    forgettable_points_ind, # which of the forgettable points we are looking at 

    """
    #sample_ind = forgettable_points[forgettable_points_ind]
    #oracle_count, N = training_masks.shape
    unlearning_count = all_unlearned_margins_ulira.shape[0]  # 1000
    target_count = unlearning_count // 2  # 500
    target_count = 50
    #shadow_count = int(0.1* unlearning_count)  # 500

    # target models are first 500 unlearned models
    target_unlearning_margins = all_unlearned_margins_ulira[:target_count]
    target_oracle_margins = all_oracle_margins_ulira[:target_count]
    target_forget_masks = forget_masks[:target_count]
    target_training_masks = training_masks[:target_count]
    ###

    shadow_unlearning_margins = all_unlearned_margins_ulira[
        target_count:unlearning_count]
    shadow_oracle_margins = all_oracle_margins_ulira[
        target_count:unlearning_count]
    shadow_forget_masks = forget_masks[target_count:unlearning_count]
    shadow_training_masks = training_masks[target_count:unlearning_count]
    if False:
        # Print statements to show shapes and sizes
        print("all_unlearned_margins_ulira shape:",
              all_unlearned_margins_ulira.shape)
        print(f"unlearning_count- {unlearning_count}")
        print("all_oracle_margins_ulira shape:",
              all_oracle_margins_ulira.shape)
        print("forget_masks shape:", forget_masks.shape)
        print("training_masks shape:", training_masks.shape)

        print("unlearning_count:", unlearning_count)
        print("shadow_count:", target_count)

        print("target_unlearning_margins shape:",
              target_unlearning_margins.shape)
        print("target_oracle_margins shape:", target_oracle_margins.shape)
        print("target_forget_masks shape:", target_forget_masks.shape)
        print("target_training_masks shape:", target_training_masks.shape)

        print("shadow_unlearning_margins shape:",
              shadow_unlearning_margins.shape)
        print("shadow_oracle_margins shape:", shadow_oracle_margins.shape)
        print("shadow_forget_masks shape:", shadow_forget_masks.shape)
        print("shadow_training_masks shape:", shadow_training_masks.shape)
        return
    #print(f"shadow_forget_masks.shape - {shadow_forget_masks.shape}")
    #print(f"target_forget_masks - {target_forget_masks.shape}")

    # get the set of indices that models consider forgetting
    forgettable_indices = list(sorted(get_forgettable_indices(forget_masks)))

    model_scores = []
    model_count = len(target_unlearning_margins)
    if max_models is not None:
        model_count = max_models

    for model_i in range(model_count):
        print(f"model {model_i}")
        target_margins = target_unlearning_margins[model_i]

        # indices that model_i forgot
        #forget_indices = target_forget_masks[model_i].nonzero()[0]
        target_forget_set_mask = target_forget_masks[model_i]
        target_training_set_mask = target_training_masks[model_i]

        forget_mask_ = np.array(target_forget_set_mask *
                                target_training_set_mask,
                                dtype=bool)
        forget_indices = forget_mask_.nonzero()[0]
        forget_pt_count = len(forget_indices)
        
        
        test_indices = get_test_points_for_ulira(
            forget_indices,
            forgettable_indices,
            num_test_points=forget_pt_count,
            seed=42)

        # take 200 points that the model never saw
        #target_training_set_indices = target_training_set_mask.nonzero()[0]
        #print(target_training_set_mask)
        #target_unseen_training_mask = 1- target_training_set_mask
        seen_training_indices = target_training_set_mask.nonzero()[0]
        #unseen_training_indices = target_unseen_training_mask.nonzero()[0]
        # test indiices is forgettable indices that have have never been seen by the model
        test_indices = list(
            set(forgettable_indices) - set(seen_training_indices))
        test_indices = sorted(list(test_indices))[:forget_pt_count]
        if verbose:
            print(f"we want a 0")

        if model_i < 3:
            plot_index = plot_index
        else:
            plot_index = -1

        # NOTE -  we don't want oracles. we want shadow models that never saw the point
        #
        test_ulira_correct = do_ulira_on_one_model(test_indices,
                                                   0,
                                                   target_margins,
                                                   shadow_unlearning_margins,
                                                   shadow_oracle_margins,
                                                   shadow_forget_masks,
                                                   shadow_training_masks,
                                                   plot_index=plot_index,
                                                   verbose=verbose)
        if verbose:
            print(f"we want a 1")
        # forget indices
        forget_ulira_correct = do_ulira_on_one_model(forget_indices,
                                                     1,
                                                     target_margins,
                                                     shadow_unlearning_margins,
                                                     shadow_oracle_margins,
                                                     shadow_forget_masks,
                                                     shadow_training_masks,
                                                     plot_index=plot_index)

        model_score = (test_ulira_correct + forget_ulira_correct) / (
            len(test_indices) + len(forget_indices))
        model_scores.append(model_score)
        if verbose:
            print(f"test_ulira_correct- {test_ulira_correct} ||")
        print(f"model - score: {model_score} - mean : {np.mean(model_scores)}")
    #raise
    return (model_scores)


noop = True# False
if noop:
    #forget_masks = ulira.get_ulira_forget_mask(ds_name, class_5_range=1000, overwrite=False )
    #forget_masks_ = generate_ulira_forget_mask(dataset_name, class_5_range=5000, training_mask=training_masks)
    print(f"do nothing")

    m, n = non_repeated_training_masks.shape
    num_models = non_repeated_all_oracle_margins_ulira.shape[0]
    forgettable_masks = np.zeros((n))
    forgettable_masks.shape
    forgettable_masks[forgettable_indices] = 1

    fake_forget_masks = []
    print(f"creating a fake mask ")

    for training_mask in non_repeated_training_masks[:num_models]:
        # pick 200 forgettable indices
        forgettable_indices_200 = np.random.choice(forgettable_indices,
                                                   200,
                                                   replace=False)
        forgettable_mask = forgettable_masks * training_mask

        # if we want to only take 200 of these points
        if False:
            forgettable_mask_indices = forgettable_mask.nonzero()[0]
            forgettable_mask_indices = np.random.choice(
                forgettable_mask_indices, 200, replace=False)
            forgettable_mask = np.zeros_like(training_mask)
            forgettable_mask[forgettable_mask_indices] = 1

        fake_forget_masks.append(forgettable_mask)
    fake_forget_masks = np.array(fake_forget_masks)
    print(f"fake_forget_masks - {fake_forget_masks.shape}")

    model_scores = ulira_paper(non_repeated_all_oracle_margins_ulira,
                                   non_repeated_all_oracle_margins_ulira,
                                   fake_forget_masks,
                                   non_repeated_training_masks,
                                   max_models=50,
                                   plot_index=2,
                                   verbose=False)

else:
    model_scores = ulira_paper(all_unlearned_margins_ulira,
                               all_oracle_margins_ulira,
                               forget_masks,
                               training_masks,
                               plot_index=2,
                               max_models=3,
                               verbose=False)
# 48883

In [ ]:
# try to load ulira model 1
def do_nothing(*args, **kwargs):
    return

In [ ]:
    # resnet 9!
from torchvision import models as torchvision_models
import torch 
from unlearning.models.resnet9 import WrappedModel
# resnet 18
model = torchvision_models.resnet18(num_classes=10)
#model = ResNet9(num_classes=10)
model= WrappedModel(model).cuda()
#model = model.load_state_dict(torch.load(ulira_model_ckpt_paths[0]))

unlearnings_per_model = 40
print(f"Note! we do {unlearnings_per_model} unlearnings per model! (this is hardcoded)")

all_ulira_forget_masks =get_ulira_forget_masks(ds_name, original_model_count = 256, class_5_range=1000, unlearnings_per_model = unlearnings_per_model , overwrite = False )
i = 40

original_model_i = i // unlearnings_per_model
forget_index = i % unlearnings_per_model

forget_set_mask = all_ulira_forget_masks[original_model_i][forget_index]
# forget_set should be the set intersection of the forget mask and the ulira mask at index i

ulira_mask = ulira.get_ulira_masks()

training_set_mask = ulira_mask[i]

#forget_set_mask = ulira_forget_mask[i]
print(f"forget_set_mask= {forget_set_mask.shape}")
print(f"training_set_mask= {training_set_mask.shape}")
print(f"type - {type(forget_set_mask)}")
print(f"type - {type(training_set_mask)}")
# bitwise and the two masks
train_and_forget = np.array(forget_set_mask * training_set_mask, dtype=bool)
#train_and_forget = np.dot(forget_set_mask, training_set_mask)
print(f"number of forget points (real): {train_and_forget.sum()}")
print(f"train_and_forget - {train_and_forget}")

forget_set_indices = (train_and_forget).nonzero()[0]

print(f"forget_set_indices- {forget_set_indices}")
models_dir = Path(
    "/n/home04/rrinberg/data_dir__holylabs/unlearning/precomputed_models/ULIRA_clean" ) / "final_models"




ulira_model_ckpt_paths = [models_dir/ f"sd_1_epoch_1.pt" ]


ulira_model_ckpt_path = ulira_model_ckpt_paths[original_model_i]
print(f"original_model_i-   {original_model_i}")
from unlearning.auditors.direct import (
    get_u_margins,
)


train_loader = loader_factory(ds_name, indexed=True)
val_loader = loader_factory(ds_name, split="val", indexed=True)
eval_set_inds = np.arange(
    len(train_loader.dataset) + len(val_loader.dataset))

eval_loader = loader_factory(ds_name,
                                split="train_and_val",
                                indices=eval_set_inds,
                                indexed=True)



_m = get_u_margins(
    model,
    ulira_model_ckpt_path,
    do_nothing,
    {},
    train_loader,
    eval_loader,
    forget_set_indices,
)

In [ ]:
training_masks.shape

In [ ]:
#np.mean((0.8125, 0.8475,0.8425, 0.805, 0.785, 0.8325, 0.82, 0.8225, 0.7825, 0.83, 0.79))

In [ ]:
splits = ["train", "val"]

ulira_model_ckpt_paths, _, _ = ulira.get_ulira_model_paths(ds_name,
                                                           splits=splits)
ulira_model_ckpt_paths[-1]

MASK_PATH = Path(
    f"/n/home04/rrinberg/data_dir__holylabs/unlearning/precomputed_models/ULIRA_clean/training_masks.npy"
)
m = np.load(MASK_PATH)
m.shape

In [ ]:
sum(m[0])


In [ ]:
def sanity_check_1(all_unlearned_margins_ulira: ch.Tensor,
                   all_oracle_margins_ulira: ch.Tensor,
                   forget_masks,
                   training_masks,
                   plot_index=-1):
    """
    forgettable_points_ind, # which of the forgettable points we are looking at 

    """
    #sample_ind = forgettable_points[forgettable_points_ind]
    #oracle_count, N = training_masks.shape
    unlearning_count = all_unlearned_margins_ulira.shape[0]  # 1000
    shadow_count = unlearning_count // 2  # 500

    # target models are first 500 unlearned models
    target_unlearning_margins = all_unlearned_margins_ulira[:shadow_count]
    target_oracle_margins = all_oracle_margins_ulira[:shadow_count]
    target_forget_masks = forget_masks[:shadow_count]
    target_training_masks = training_masks[:shadow_count]
    ###

    shadow_unlearning_margins = all_unlearned_margins_ulira[
        shadow_count:unlearning_count]
    shadow_oracle_margins = all_oracle_margins_ulira[
        shadow_count:unlearning_count]
    shadow_forget_masks = forget_masks[shadow_count:unlearning_count]
    shadow_training_masks = training_masks[shadow_count:unlearning_count]

    #print(f"shadow_forget_masks.shape - {shadow_forget_masks.shape}")
    #print(f"target_forget_masks - {target_forget_masks.shape}")

    # get the set of indices that models consider forgetting
    forgettable_indices = get_forgettable_indices(forget_masks)
    model_scores = []
    model_count = len(target_unlearning_margins)
    model_count = 4
    for model_i in range(model_count):
        print(f"model {model_i}")
        # margins of model i
        target_margins = target_unlearning_margins[model_i]

        # indices that model_i forgot
        forget_indices = target_forget_masks[model_i].nonzero()[0]
        forget_pt_count = len(forget_indices)

        test_indices = get_test_points_for_ulira(
            forget_indices,
            forgettable_indices,
            num_test_points=forget_pt_count,
            seed=42)


        target_forget_set_mask = target_forget_masks[model_i]
        target_training_set_mask = target_training_masks[model_i]

        forget_mask_ = np.array(target_forget_set_mask *
                                target_training_set_mask,
                                dtype=bool)
        forget_indices = forget_mask_.nonzero()[0]
        forget_pt_count = len(forget_indices)
        
        
        test_indices = get_test_points_for_ulira(
            forget_indices,
            forgettable_indices,
            num_test_points=forget_pt_count,
            seed=42)

        seen_training_indices = target_training_set_mask.nonzero()[0]
        test_indices = list(
            set(forgettable_indices) - set(seen_training_indices))
        test_indices = sorted(list(test_indices))[:forget_pt_count]



        # plot histogram of margins in test and forget
        plt.title(f"model {model_i}")
        plt.hist(target_margins[forget_indices],
                 bins=50,
                 alpha=0.5,
                 label="forget")
        plt.hist(target_margins[test_indices],
                 bins=50,
                 alpha=0.5,
                 label="test")
        plt.legend()
        plt.show()
    return (model_scores)

print(f"this is plotting - model 0 on the forget set vs the test set")
model_scores = sanity_check_1(all_unlearned_margins_ulira,
                            all_oracle_margins_ulira,
                            forget_masks,
                            training_masks,
                            plot_index=0)

In [ ]:
def sanity_check_2(all_unlearned_margins_ulira: ch.Tensor,
                   all_oracle_margins_ulira: ch.Tensor,
                   forget_masks,
                   training_masks,
                   plot_index=-1):
    """
    forgettable_points_ind, # which of the forgettable points we are looking at 

    """
    #sample_ind = forgettable_points[forgettable_points_ind]
    #oracle_count, N = training_masks.shape
    unlearning_count = all_unlearned_margins_ulira.shape[0]  # 1000
    shadow_count = unlearning_count // 2  # 500

    # target models are first 500 unlearned models
    target_unlearning_margins = all_unlearned_margins_ulira[:shadow_count]
    target_oracle_margins = all_oracle_margins_ulira[:shadow_count]
    target_forget_masks = forget_masks[:shadow_count]
    target_training_masks = training_masks[:shadow_count]
    ###

    shadow_unlearning_margins = all_unlearned_margins_ulira[
        shadow_count:unlearning_count]
    shadow_oracle_margins = all_oracle_margins_ulira[
        shadow_count:unlearning_count]
    shadow_forget_masks = forget_masks[shadow_count:unlearning_count]
    shadow_training_masks = training_masks[shadow_count:unlearning_count]

    #print(f"shadow_forget_masks.shape - {shadow_forget_masks.shape}")
    #print(f"target_forget_masks - {target_forget_masks.shape}")

    # get the set of indices that models consider forgetting
    forgettable_indices = get_forgettable_indices(forget_masks)
    model_scores = []
    model_count = len(target_unlearning_margins)
    model_count = 4
    points_of_interest = forgettable_indices[:model_count]
    for point_of_interest in points_of_interest:
        # models that never saw points_of_interest
        models_without_x, _ = get_training_models_with_and_without(
            target_training_masks, point_of_interest)
        # models that forgot points_of_interest
        models_with_x_forgotten = get_models_with_pt_forgotten(
            target_training_masks, target_forget_masks, point_of_interest)
        # models that retained x
        models_with_x_retained = get_models_with_pt_retained(
            target_training_masks, target_forget_masks, point_of_interest)

        unlearned_margins_of_interest = target_unlearning_margins[:,
                                                                  point_of_interest]
        oracle_margins_of_interest = target_oracle_margins[:,
                                                           point_of_interest]

        untrained_margins = oracle_margins_of_interest[models_without_x]
        forgotten_margins = unlearned_margins_of_interest[models_with_x_forgotten]
        retained_margins = oracle_margins_of_interest[models_with_x_retained]

        # plot histogram of margins in test and forget
        plt.title(f"pt {point_of_interest}")
        density = True
        plt.hist(untrained_margins,
                 bins=50,
                 alpha=0.5,
                 label="untrained",
                 density=density)
        plt.hist(forgotten_margins,
                 bins=50,
                 alpha=0.5,
                 label="forgotten",
                 density=density)
        plt.hist(retained_margins,
                 bins=50,
                 alpha=0.5,
                 label="retained",
                 density=density)
        plt.legend()
        plt.show()
    return


model_scores = sanity_check_2(all_unlearned_margins_ulira,
                            all_oracle_margins_ulira,
                            forget_masks,
                            training_masks,
                            plot_index=0)

In [ ]:
def computation_round_2(
    all_unlearned_margins_ulira: ch.Tensor,
    all_oracle_margins_ulira: ch.Tensor,
    forget_masks,
    training_masks,
):
    """
    forgettable_points_ind, # which of the forgettable points we are looking at 

    """
    #sample_ind = forgettable_points[forgettable_points_ind]
    #oracle_count, N = training_masks.shape
    unlearning_count = all_unlearned_margins_ulira.shape[0]  # 1000
    shadow_count = unlearning_count // 2  # 500

    # target models are first 500 unlearned models
    target_unlearning_margins = all_unlearned_margins_ulira[:shadow_count]
    target_oracle_margins = all_oracle_margins_ulira[:shadow_count]
    target_forget_masks = forget_masks[:shadow_count]
    target_training_masks = training_masks[:shadow_count]
    ###

    shadow_unlearning_margins = all_unlearned_margins_ulira[
        shadow_count:unlearning_count]
    shadow_oracle_margins = all_oracle_margins_ulira[
        shadow_count:unlearning_count]
    shadow_forget_masks = forget_masks[shadow_count:unlearning_count]
    shadow_training_masks = training_masks[shadow_count:unlearning_count]

    #print(f"shadow_forget_masks.shape - {shadow_forget_masks.shape}")
    #print(f"target_forget_masks - {target_forget_masks.shape}")

    # get the set of indices that models consider forgetting
    forgettable_indices = get_forgettable_indices(forget_masks)

    target_model_ind = 1
    target_model_margins = target_unlearning_margins[target_model_ind]
    # is plot_index in forget point for index 0
    forget_indices = target_forget_masks[target_model_ind].nonzero()[0]
    # forgettable_indices - forget_indices
    retain_indices = list(set(forgettable_indices) - set(forget_indices))

    # indices to plot
    num_indices = 10
    plot_indices = list(forget_indices[:num_indices]) + list(
        retain_indices[:num_indices])
    #print(f"plot_indices- {plot_indices}")
    #plot_index = 3
    #plot_indices = forgettable_indices[:10]

    #plot_indices = [851, 195]
    #plot_indices = [48883]
    total_correct = 0
    for plot_index in plot_indices:  #[0,-1, forgettable_indices[0], forgettable_indices[1], forgettable_indices[100]]:
        # get target_models
        # indices that model_i forgot
        # models with index in forget_set, and models without
        target_models_with_index, target_models_without_index = get_training_models_with_and_without(
            target_training_masks, plot_index)

        target_margins_with = target_unlearning_margins[
            target_models_with_index]
        target_margins_without = target_unlearning_margins[
            target_models_without_index]

        shadow_models_with_index, shadow_models_without_index = get_training_models_with_and_without(
            shadow_training_masks, plot_index)
        shadow_margins_with = shadow_unlearning_margins[
            shadow_models_with_index]
        shadow_margins_without = shadow_unlearning_margins[
            shadow_models_without_index]

        # get models that forgot the point
        target_models_forgot_indices = get_models_with_pt_forgotten(
            target_training_masks, target_forget_masks, plot_index)

        target_margins_forget = target_unlearning_margins[
            target_models_forgot_indices]
        shadow_models_forgot_indices = get_models_with_pt_forgotten(
            shadow_training_masks, shadow_forget_masks, plot_index)
        shadow_margins_forget = shadow_unlearning_margins[
            shadow_models_forgot_indices]

        ###

        def get_gaussian_at_point(pts, ind):
            pts_ = pts[:, ind].cpu().numpy()
            mean, std = np.mean(pts_), np.std(pts_)
            x = target_model_margins[ind]
            return gaussian_pdf(x, mean, std)

        forget_likelihood = get_gaussian_at_point(shadow_margins_forget,
                                                  plot_index)
        oracle_likelihood = get_gaussian_at_point(shadow_oracle_margins,
                                                  plot_index)
        likelihood_ratio = forget_likelihood / (forget_likelihood +
                                                oracle_likelihood)
        #cast likelihood_ratio to float
        likelihood_ratio = float(likelihood_ratio)
        #print(f"likelihood_ratio {type(likelihood_ratio)}")
        ulira_label = int(likelihood_ratio > 0.5)

        included_in_forget = plot_index in forget_indices

        correct = ulira_label == included_in_forget
        print(
            f"did it get it correct- {correct}? | true label - {included_in_forget} | ulira label - {ulira_label}"
        )
        total_correct += int(correct)
        if True:
            #####
            density = True
            plt.title(
                f"margins at index {plot_index} - {round(likelihood_ratio,2)} | correct {correct}"
            )

            def plot_hist(pts, ind, label):
                pts_ = pts[:, ind].cpu().numpy()
                mean, std = np.mean(pts_), np.std(pts_)
                # plt guassian
                x = np.linspace(-20, 20, 1000)
                plt.plot(x, gaussian_pdf(x, mean, std), label=label)
                # plts hist

                # get prev color
                c = plt.gca().lines[-1].get_color()

                plt.hist(
                    pts_,
                    bins=25,
                    alpha=0.5,
                    #label=label,
                    color=c,
                    density=density)

            # forget points
            plot_hist(target_margins_forget, plot_index, "target forget")
            plot_hist(shadow_margins_forget, plot_index, "shadow forget")
            bin_count = 25
            # test points
            plot_hist(target_margins_with, plot_index, "target with")
            plot_hist(shadow_margins_with, plot_index, "shadow with")
            x = target_model_margins[plot_index]
            plt.axvline(x, color="black", linestyle="--")
            plt.xlim(-20, 20)
            plt.legend()
            plt.show()
    print(f"total_correct - {total_correct} / {len(plot_indices)}")
computation_round_2(
    all_unlearned_margins_ulira,
    all_oracle_margins_ulira,
    forget_masks,
    training_masks,
)

In [ ]:
def plotting_investigation(
    all_unlearned_margins_ulira: ch.Tensor,
    all_oracle_margins_ulira: ch.Tensor,
    forget_masks,
    training_masks,
):
    """
    forgettable_points_ind, # which of the forgettable points we are looking at 

    """
    #sample_ind = forgettable_points[forgettable_points_ind]
    #oracle_count, N = training_masks.shape
    unlearning_count = all_unlearned_margins_ulira.shape[0]  # 1000
    shadow_count = unlearning_count // 2  # 500

    # target models are first 500 unlearned models
    target_unlearning_margins = all_unlearned_margins_ulira[:shadow_count]
    target_oracle_margins = all_oracle_margins_ulira[:shadow_count]
    target_forget_masks = forget_masks[:shadow_count]
    target_training_masks = training_masks[:shadow_count]
    ###

    shadow_unlearning_margins = all_unlearned_margins_ulira[
        shadow_count:unlearning_count]
    shadow_oracle_margins = all_oracle_margins_ulira[
        shadow_count:unlearning_count]
    shadow_forget_masks = forget_masks[shadow_count:unlearning_count]
    shadow_training_masks = training_masks[shadow_count:unlearning_count]

    #print(f"shadow_forget_masks.shape - {shadow_forget_masks.shape}")
    #print(f"target_forget_masks - {target_forget_masks.shape}")

    # get the set of indices that models consider forgetting
    forgettable_indices = get_forgettable_indices(forget_masks)

    target_model_ind = 1
    target_model_margins = target_unlearning_margins[target_model_ind]
    # is plot_index in forget point for index 0
    forget_indices = target_forget_masks[target_model_ind].nonzero()[0]
    # forgettable_indices - forget_indices
    retain_indices = list(set(forgettable_indices) - set(forget_indices))

    # indices to plot
    plot_indices = list(forget_indices[:4]) + list(retain_indices[:4])
    #print(f"plot_indices- {plot_indices}")
    #plot_index = 3
    #plot_indices = forgettable_indices[:10]

    #plot_indices = [851, 195]
    #plot_indices = [48883]
    total_correct = 0
    for plot_index in plot_indices:  #[0,-1, forgettable_indices[0], forgettable_indices[1], forgettable_indices[100]]:
        # get target_models
        # indices that model_i forgot
        # models with index in forget_set, and models without
        target_models_with_index, target_models_without_index = get_training_models_with_and_without(
            target_training_masks, plot_index)

        target_margins_with = target_unlearning_margins[
            target_models_with_index]
        target_margins_without = target_unlearning_margins[
            target_models_without_index]

        shadow_models_with_index, shadow_models_without_index = get_training_models_with_and_without(
            shadow_training_masks, plot_index)
        shadow_margins_with = shadow_unlearning_margins[
            shadow_models_with_index]
        shadow_margins_without = shadow_unlearning_margins[
            shadow_models_without_index]

        # get models that forgot the point
        target_models_forgot_indices = get_models_with_pt_forgotten(
            target_training_masks, target_forget_masks, plot_index)

        target_margins_forget = target_unlearning_margins[
            target_models_forgot_indices]
        shadow_models_forgot_indices = get_models_with_pt_forgotten(
            shadow_training_masks, shadow_forget_masks, plot_index)
        shadow_margins_forget = shadow_unlearning_margins[
            shadow_models_forgot_indices]
        ###

        ####
        included_in_forget = plot_index in forget_indices
        # shadow_models_with_index
        # shadow_models_forgot_indices
        # could be that model_i is inccorectly index
        print(f"it is included! - {included_in_forget}")
        test_ulira_correct = do_ulira_on_one_model([plot_index],
                                                   included_in_forget,
                                                   target_model_margins,
                                                   shadow_unlearning_margins,
                                                   #shadow_oracle_margins,
                                                   shadow_forget_masks,
                                                   shadow_training_masks,
                                                   plot_index=0)
        density = True
        correct = test_ulira_correct == included_in_forget
        print(f"did it get it correct- {correct}?")
        total_correct += int(correct)
        #####
        plt.title(
            f"margins at index {plot_index} | model label: {test_ulira_correct}"
        )

        def plot_hist(pts, ind, label):
            pts_ = pts[:, ind].cpu().numpy()
            mean, std = np.mean(pts_), np.std(pts_)
            # plt guassian
            x = np.linspace(-20, 20, 1000)
            plt.plot(x, gaussian_pdf(x, mean, std), label=label)
            # plts hist
            # get prev color
            c = plt.gca().lines[-1].get_color()

            plt.hist(
                pts_,
                bins=25,
                alpha=0.5,
                #label=label,
                color=c,
                density=density)

        # forget points
        plot_hist(target_margins_forget, plot_index, "target forget")
        plot_hist(shadow_margins_forget, plot_index, "shadow forget")
        if False:
            plt.hist(target_margins_forget[:, plot_index].cpu().numpy(),
                     bins=25,
                     alpha=0.5,
                     label="target forget",
                     density=density)
            plt.hist(shadow_margins_forget[:, plot_index].cpu().numpy(),
                     bins=25,
                     alpha=0.5,
                     label="shadow forget",
                     density=density)
        ####
        bin_count = 25
        # test points
        plot_hist(target_margins_with, plot_index, "target with")
        plot_hist(shadow_margins_with, plot_index, "shadow with")
        if False:
            plt.hist(target_margins_without[:, plot_index].cpu().numpy(),
                     bins=bin_count,
                     alpha=0.5,
                     label="target without",
                     density=density)
            plt.hist(shadow_margins_without[:, plot_index].cpu().numpy(),
                     bins=bin_count,
                     alpha=0.5,
                     label="shadow without",
                     density=density)
        plt.xlim(-20, 20)
        plt.legend()
        plt.show()
    print(f"total_correct - {total_correct} / {len(plot_indices)}")
plotting_investigation(
    all_unlearned_margins_ulira,
    all_oracle_margins_ulira,
    forget_masks,
    training_masks,
)


In [ ]:
# my confusion is 

In [ ]:
mean = np.mean(forget_ulira_scores)
std = np.std(forget_ulira_scores)
method_name = "GA"
plt.hist(forget_ulira_scores)
plt.title(f"ULIRA scores - {method_name}")
plt.xlabel("ULIRA score")
plt.ylabel("Frequency")
# vert line at mean
plt.axvline(mean, color='k', linestyle='dashed', linewidth=1)
plt.show()

In [ ]:
forget_masks[:4].shape

In [ ]:
training_masks.shape
forget_masks.shape

In [ ]:
print(f"fall_unlearned_margins_ulira. - {all_unlearned_margins_ulira.shape }") # 1000 x N 
print(f"all_oracle_margins_ulira. - {all_oracle_margins_ulira.shape }") # 2000 x N

In [ ]:
import re
import numpy as np
import torch as ch
from tqdm import tqdm
from pathlib import Path
from typing import List, Optional

from unlearning.unlearning_algos.utils import get_margin
from unlearning.auditors.direct import (
    direct_audit_precomputed,
    get_u_margins,
    u_margin_job,
    plot_margins_direct,
)
from scipy.stats import norm
import torch
from unlearning.auditors.utils import (
    loader_factory,
 )


import os
import shutil
import yaml
import numpy as np
import torch as ch
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from pathlib import Path
import logging
import pprint
from contextlib import redirect_stdout, redirect_stderr

from unlearning.auditors.utils import (
    model_factory,
    loader_factory,
    load_forget_set_indices,
    get_full_model_paths,

    get_oracle_paths,
    make_results_dir,
)
from unlearning.auditors.accuracies import eval_accuracy
from unlearning.auditors.logit_plots import compute_logits, plot_logits
from unlearning.auditors.basic import plot_margins

from unlearning.auditors.direct import (
    config_submitit,
    direct_audit_precomputed,
    get_u_margins,
    u_margin_job,
    plot_margins_direct,
)
from unlearning.auditors import ulira
from unlearning.unlearning_algos.base_nn import NAME_TO_ALGO


from contextlib import redirect_stdout, redirect_stderr


# 1. fix ulira accuracy measure! get test scores too !
# 2.  
# 

# load model from chkpt and evaluate accuracy 


def _model_factory(dataset, wrapped=True):
    """
    for now, let's tie the model to the dataset, so we have fewer moving pieces
    """
    if dataset.lower() == "cifar10":
        from unlearning.models.resnet9 import ResNet9

        return ResNet9(num_classes=10, wrapped=wrapped)

    elif dataset.lower() == "cifar100":
        from unlearning.models.resnet9 import ResNet9

        return ResNet9(num_classes=100)
    else:
        raise NotImplementedError

ds_name = "CIFAR10"
# for now, let's tie the model to the dataset, so we have fewer moving pieces
model = _model_factory(ds_name)  # on cuda, in eval mode
model

with redirect_stdout(open("/dev/null", "w")):
    # no shuffling, no augmentation
    train_loader = loader_factory(ds_name, indexed=True)
    val_loader = loader_factory(ds_name, split="val", indexed=True)
    eval_set_inds = np.arange(
        len(train_loader.dataset) + len(val_loader.dataset))
    eval_loader = loader_factory(ds_name,
                                    split="train_and_val",
                                    indices=eval_set_inds,
                                    indexed=True)
    
splits = ["train", "val"]
ulira_model_ckpt_paths, _, _ = ulira.get_ulira_model_paths(ds_name,
                                                            splits=splits)
#ulira_mask = ulira.get_ulira_masks(ds_name)
#ulira_forget_mask = ulira.get_ulira_forget_mask(ds_name)

#logger.info(f"Loaded paths of pretrained models.")

model = load_model(ulira_model_ckpt_paths[0], _model_factory, ds_name)

#logger.info(f"Loaded a pretrained model.")
####### END OF LOADING PRETRAINED MODELS ########

model

In [ ]:
# working version:
# get 200 points from class 5 that are not from the forgettable_points
#forgettable_indices
def get_test_points_for_ulira(forget_indices, forgettable_indices, num_test_points=200, seed = 42):
    """
    get 200 points forgettable points from class 5 that are not in the forget_indices
    """
    np.random.seed(seed)
    # copy forgettable_indices
    class_5 = forgettable_indices.copy()
    class_5 = np.array(list(set(class_5) - set(forget_indices)))
    np.random.shuffle(class_5)
    return list(class_5)[:num_test_points]

def gaussian_pdf(x, mean, std):
    return 1 / (std * np.sqrt(2 * np.pi)) * np.exp(-0.5 * ((x - mean) / std) ** 2)


def single_ulira(
        all_unlearned_margins: ch.Tensor,  # T x n
        all_test_margins: ch.Tensor,  # T x n
        unlearned_model_margin: float,
        sample_ind,
        threshold=0.5,
        plot=False, verbose= False ):
    """
    Compute the ULIRA (Unlearning Likelihood Ratio) results for each sample. for a single unlearned model.

    Note: 
        1: (x,y) is likely a member of training (unlearned model)
        0: (x,y) is likely a member of the oracle (oracle model)

    Args:
        all_unlearned_margins (ch.Tensor): Tensor of shape T x n containing the margins of all unlearned models.
        all_oracle_margins (ch.Tensor): Tensor of shape T x n containing the margins of all oracle models.
        hold_out_model (int): Index of the hold-out model.
        threshold (float, optional): Threshold value for the likelihood ratio. Defaults to 0.5.

    Returns:
        np.ndarray: Array of ULIRA results for each sample, where 1 indicates the likelihood ratio is above the threshold, and 0 otherwise.
    """
    #
    test_margins = all_test_margins[:, sample_ind].cpu().numpy()

    unlearned_margins= all_unlearned_margins[:, sample_ind].cpu().numpy()

    #
    #unlearned_model_margin = unlearned_margins[hold_out_model]
    #other_unlearned_models = np.delete(unlearned_margins, hold_out_model)

    # fit gaussians
    test_mean, test_std = np.mean(test_margins), np.std(test_margins)
    unlearned_mean, unlearned_std = np.mean(unlearned_margins), np.std(
        unlearned_margins)
    #print(f"aaa ({np.round(oracle_mean,2)}, {np.round(oracle_std,2)}) vs ({np.round(unlearned_mean,2)}, {np.round(unlearned_std,2)})")
    # compute LIRA
    unlearned_model_margin = float(unlearned_model_margin)

    oracle_prob = gaussian_pdf(unlearned_model_margin, test_mean, test_std)
    unlearned_prob = gaussian_pdf(unlearned_model_margin, unlearned_mean,
                                  unlearned_std)

    likelihood_ratio = unlearned_prob / (unlearned_prob + oracle_prob)

    # print(f"likelihood_ratio - {likelihood_ratio} : type - {type(likelihood_ratio)}")

    likelihood_ratio = float(likelihood_ratio)
    if verbose:
        print(f"unlearned_margins shape - {unlearned_margins.shape}")
        print(f"oracle_margins shape - {test_margins.shape}")
        print(f"oracle_margins - {test_margins}")
        
    if plot:
        # plot gaussians
        plt.xlim(-20, 20)
        x = np.linspace(-20, 20, 1000)
        plt.hist(test_margins,
                 bins=50,
                 alpha=0.5,
                 label="oracle",
                 density=True)
        plt.hist(unlearned_margins,
                 bins=50,
                 alpha=0.5,
                 label="unlearned",
                 density=True)
        oracle_pdf = gaussian_pdf(x, test_mean, test_std)
        unlearned_pdf = gaussian_pdf(x, unlearned_mean, unlearned_std)
        plt.title(
            f"sample index: {sample_ind} | likelihood ratio: {round(likelihood_ratio,2):.2f} = {round(unlearned_prob,2)} / ({round(unlearned_prob,2)} + {round(oracle_prob,2)})"
        )
        plt.plot(x, oracle_pdf, label="oracle")
        plt.plot(x, unlearned_pdf, label="unlearned")
        plt.axvline(unlearned_model_margin, color="black", linestyle="--")
        plt.legend()
        plt.show()
    # save result
    return float(likelihood_ratio > threshold)



#####
def get_training_models_with_and_without(masks, index):
    """
    return indices of models with and without the point
    """
    m = masks[:, index].T

    with_index, without_index = np.where(m == 1)[0], np.where(m == 0)[0]
    return with_index, without_index



def get_models_with_pt_forgotten(masks, forget_mask, sample_index):
    """
    find the indices where the point is in the plan and in the forget mask
    """

    models_with = []
    models_inds_with, _ = get_training_models_with_and_without(
    masks, sample_index)

    for model_ind in models_inds_with:
        if forget_mask[model_ind, sample_index] == 1:
            models_with.append(model_ind)
    return np.array(models_with)

def get_models_with_pt_retained(masks, forget_mask, sample_index):
    """
    find the indices where the point is in the plan and in the forget mask
    """

    models_with = []
    models_inds_with, _ = get_training_models_with_and_without(
    masks, sample_index)

    for model_ind in models_inds_with:
        if forget_mask[model_ind, sample_index] == 0:
            models_with.append(model_ind)
    return np.array(models_with)
#####

def do_ulira_on_one_model(indices,
                          correct_ulira_label,
                          target_margins,
                          shadow_unlearning_margins,
                          shadow_oracle_margins,
                          forget_masks,
                          training_masks,
                          plot_index=-1, verbose=  False ):
    """

    """
    correct_ulira = 0
    ulira_labels = []
    for i, sample_index in enumerate(indices):

        # get models with x forgotten and models that never saw x
        models_with_x_forgotten = get_models_with_pt_forgotten(
            training_masks, forget_masks, sample_index)
        _, models_without_x = get_training_models_with_and_without(
            training_masks, sample_index)

        num_models_with_x_forgotten = len(models_with_x_forgotten)
        num_models_without_x = len(models_without_x)
        if verbose:
            print(f"num_models_with_x_forgotten - {num_models_with_x_forgotten}")
            print(f"num_models_without_x - {num_models_without_x}")

        max_models = 300

        min_models = min(num_models_with_x_forgotten, num_models_without_x)
        min_models = min(max_models, min_models)
        #
        #print(f"min models - {min_models    }")
        #min_models = -1
        np.random.shuffle(unlearned_models_with_x_forgotten)
        np.random.shuffle(models_without_x)

        models_with_x_forgotten = models_with_x_forgotten[:min_models]
        models_without_x = models_without_x[:min_models]

        # the margin of the target model on sample index
        target_margin = target_margins[sample_index]

        # we want to do a lira test between unlearned models that saw x
        # and unlearned models that did not see x

        ulira_label = single_ulira(
            shadow_unlearning_margins[models_with_x_forgotten],
            shadow_unlearning_margins[models_without_x],
            target_margin,
            sample_index,
            plot=(i == plot_index), verbose= verbose)
        if False:
            ulira_label = single_ulira(
                shadow_unlearning_margins[models_with_x_forgotten],
                shadow_oracle_margins[models_without_x],
                target_margin,
                sample_index,
                plot=(i == plot_index))
        ulira_labels.append(ulira_label)
        if ulira_label == correct_ulira_label:
            correct_ulira += 1
    return correct_ulira



def ulira_paper(all_unlearned_margins_ulira: ch.Tensor,
                all_oracle_margins_ulira: ch.Tensor,
                forget_masks,
                training_masks,
                plot_index=-1,
                verbose=False,
                max_models=None):
    """
    forgettable_points_ind, # which of the forgettable points we are looking at 

    """
    #sample_ind = forgettable_points[forgettable_points_ind]
    #oracle_count, N = training_masks.shape
    unlearning_count = all_unlearned_margins_ulira.shape[0]  # 1000
    shadow_count = unlearning_count // 2  # 500

    # target models are first 500 unlearned models
    target_unlearning_margins = all_unlearned_margins_ulira[:shadow_count]
    target_oracle_margins = all_oracle_margins_ulira[:shadow_count]
    target_forget_masks = forget_masks[:shadow_count]
    target_training_masks = training_masks[:shadow_count]
    ###

    shadow_unlearning_margins = all_unlearned_margins_ulira[
        shadow_count:unlearning_count]
    shadow_oracle_margins = all_oracle_margins_ulira[
        shadow_count:unlearning_count]
    shadow_forget_masks = forget_masks[shadow_count:unlearning_count]
    shadow_training_masks = training_masks[shadow_count:unlearning_count]
    very_verbose=  False
    if very_verbose:
        # Print statements to show shapes and sizes
        print("all_unlearned_margins_ulira shape:",
            all_unlearned_margins_ulira.shape)
        print(f"unlearning_count- {unlearning_count}")
        print("all_oracle_margins_ulira shape:", all_oracle_margins_ulira.shape)
        print("forget_masks shape:", forget_masks.shape)
        print("training_masks shape:", training_masks.shape)

        print("unlearning_count:", unlearning_count)
        print("shadow_count:", shadow_count)

        print("target_unlearning_margins shape:", target_unlearning_margins.shape)
        print("target_oracle_margins shape:", target_oracle_margins.shape)
        print("target_forget_masks shape:", target_forget_masks.shape)
        print("target_training_masks shape:", target_training_masks.shape)

        print("shadow_unlearning_margins shape:", shadow_unlearning_margins.shape)
        print("shadow_oracle_margins shape:", shadow_oracle_margins.shape)
        print("shadow_forget_masks shape:", shadow_forget_masks.shape)
        print("shadow_training_masks shape:", shadow_training_masks.shape)
        return
    #print(f"shadow_forget_masks.shape - {shadow_forget_masks.shape}")
    #print(f"target_forget_masks - {target_forget_masks.shape}")

    # get the set of indices that models consider forgetting
    forgettable_indices = list(sorted(get_forgettable_indices(forget_masks)))

    model_scores = []
    model_count = len(target_unlearning_margins)
    if max_models is not None:
        model_count = max_models

    for model_i in range(model_count):
        print(f"model {model_i}")
        target_margins = target_unlearning_margins[model_i]

        # indices that model_i forgot
        #forget_indices = target_forget_masks[model_i].nonzero()[0]
        target_forget_set_mask = target_forget_masks[model_i]
        target_training_set_mask = target_training_masks[model_i]

        forget_mask_ = np.array(target_forget_set_mask *
                                target_training_set_mask,
                                dtype=bool)
        forget_indices = forget_mask_.nonzero()[0]
        forget_pt_count = len(forget_indices)
        
        
        test_indices = get_test_points_for_ulira(
            forget_indices,
            forgettable_indices,
            num_test_points=forget_pt_count,
            seed=42)

        # take 200 points that the model never saw
        #target_training_set_indices = target_training_set_mask.nonzero()[0]
        #print(target_training_set_mask)
        #target_unseen_training_mask = 1- target_training_set_mask
        seen_training_indices = target_training_set_mask.nonzero()[0]
        #unseen_training_indices = target_unseen_training_mask.nonzero()[0]
        # test indiices is forgettable indices that have have never been seen by the model
        test_indices = list(
            set(forgettable_indices) - set(seen_training_indices))
        test_indices = sorted(list(test_indices))[:forget_pt_count]
        if verbose:
            print(f"we want a 0")

        if model_i == 0:
            plot_index = plot_index
        else:
            plot_index = -1

        # NOTE -  we don't want oracles. we want shadow models that never saw the point
        #
        test_ulira_correct = do_ulira_on_one_model(test_indices,
                                                   0,
                                                   target_margins,
                                                   shadow_unlearning_margins,
                                                   shadow_oracle_margins,
                                                   shadow_forget_masks,
                                                   shadow_training_masks,
                                                   plot_index=plot_index, verbose=verbose)
        if verbose:
            print(f"we want a 1")
        # forget indices
        forget_ulira_correct = do_ulira_on_one_model(forget_indices,
                                                     1,
                                                     target_margins,
                                                     shadow_unlearning_margins,
                                                     shadow_oracle_margins,
                                                     shadow_forget_masks,
                                                     shadow_training_masks,
                                                     plot_index=plot_index)

        model_score = (test_ulira_correct + forget_ulira_correct) / (
            len(test_indices) + len(forget_indices))
        model_scores.append(model_score)
        if verbose:
            print(f"test_ulira_correct- {test_ulira_correct} ||")
        print(f"model - score: {model_score} - mean : {np.mean(model_scores)}")
    #raise
    return (model_scores)



noop = True# False
if noop:
    #forget_masks = ulira.get_ulira_forget_mask(ds_name, class_5_range=1000, overwrite=False )
    #forget_masks_ = generate_ulira_forget_mask(dataset_name, class_5_range=5000, training_mask=training_masks)
    print(f"do nothing")
    model_scores = ulira_paper(repeated_all_oracle_margins_ulira,
                               repeated_all_oracle_margins_ulira,
                               forget_masks,
                               repeated_training_masks,
                               max_models= 1,
                               plot_index=2, verbose= True)

else:
    model_scores = ulira_paper(all_unlearned_margins_ulira,
                               all_oracle_margins_ulira,
                               forget_masks,
                               training_masks,
                               plot_index=-1,
                               max_models=100,
                               verbose=False)
# 48883